In [1]:
!pip install sentence-transformers

  Using cached sentence_transformers-2.7.0-py3-none-any.whl.metadata (11 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 3.0 MB/s eta 0:00:00a 0:00:01
  Using cached torch-2.3.0-cp312-none-macosx_11_0_arm64.whl.metadata (26 kB)
  Using cached scikit_learn-1.4.2-cp312-cp312-macosx_12_0_arm64.whl.metadata (11 kB)
  Using cached scipy-1.13.0-cp312-cp312-macosx_12_0_arm64.whl.metadata (60 kB)
  Using cached huggingface_hub-0.23.0-py3-none-any.whl.metadata (12 kB)
  Using cached filelock-3.14.0-py3-none-any.whl.metadata (2.8 kB)
  Using cached fsspec-2024.3.1-py3-none-any.whl.metadata (6.8 kB)
  Using cached sympy-1.12-py3-none-any.whl.metadata (12 kB)
  Using cached networkx-3.3-py3-none-any.whl.metadata (5.1 kB)
  Using cached tokenizers-0.19.1-cp312-cp312-macosx_11_0_arm64.whl.metadata (6.7 kB)
  Using cached safetensors-0.4.3-cp312-cp312-macosx_11_0_arm64.whl.metadata (3.8 kB)
  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
  Using cached mpmath-1.3.

In [2]:
import pandas as pd

pd.set_option("display.max_colwidth", 100)

In [4]:
df = pd.read_csv("sample_text.csv")
df.shape

(8, 2)

In [5]:
df

,text,category
0,Meditation and yoga can improve mental health,Health
1,"Fruits, whole grains and vegetables helps control blood pressure",Health
2,These are the latest fashion trends for this week,Fashion
3,Vibrant color jeans for male are becoming a trend,Fashion
4,The concert starts at 7 PM tonight,Event
5,Navaratri dandiya program at Expo center in Mumbai this october,Event
6,Exciting vacation destinations for your next trip,Travel
7,Maldives and Srilanka are gaining popularity in terms of low budget vacation places,Travel


In [6]:
df.text

0                                          Meditation and yoga can improve mental health
1                       Fruits, whole grains and vegetables helps control blood pressure
2                                      These are the latest fashion trends for this week
3                                      Vibrant color jeans for male are becoming a trend
4                                                     The concert starts at 7 PM tonight
5                        Navaratri dandiya program at Expo center in Mumbai this october
6                                      Exciting vacation destinations for your next trip
7    Maldives and Srilanka are gaining popularity in terms of low budget vacation places
Name: text, dtype: object

In [8]:
from sentence_transformers import SentenceTransformer

encoder = SentenceTransformer("all-mpnet-base-v2")
vectors = encoder.encode(df.text)
vectors.shape

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

/Users/felixachter/.pyenv/versions/3.12.2/envs/article-gpt/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

(8, 768)

In [9]:
dim = vectors.shape[1]
dim

768

In [10]:
import faiss

index = faiss.IndexFlatL2(dim)
index

<faiss.swigfaiss.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x3a94871b0> >

In [11]:
index.add(vectors)

In [32]:
search_query = "Looking for a place to visit during christmas"
search_vector = encoder.encode(search_query)
search_vector.shape

(768,)

In [33]:
import numpy as np

search_vector = np.array(search_vector).reshape(1, -1)
search_vector.shape

(1, 768)

In [34]:
distances, indices = index.search(search_vector, k=2)

In [35]:
indices.tolist()[0]

[6, 7]

In [36]:
df.loc[indices.tolist()[0]]

,text,category
6,Exciting vacation destinations for your next trip,Travel
7,Maldives and Srilanka are gaining popularity in terms of low budget vacation places,Travel


In [37]:
import os
from transformers import pipeline
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())
pipe = pipeline("text-generation", model="meta-llama/Meta-Llama-Guard-2-8B",
                token=os.getenv("HUGGINGFACEHUB_API_TOKEN"))


/Users/felixachter/.pyenv/versions/3.12.2/envs/article-gpt/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


model-00001-of-00004.safetensors:  53%|#####3    | 2.65G/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/126 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.9k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [38]:
pipe("Tell me a joke")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
/Users/felixachter/.pyenv/versions/3.12.2/envs/article-gpt/lib/python3.12/site-packages/transformers/generation/utils.py:1141: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


[{'generated_text': 'Tell me a joke, please."\n"Okay. What\'s the difference between a hippopotamus and'}]